In [ ]:
import pandas as pd
import torch
# /Users/arrao/Documents/Atchut/Book-PPML/Chapter6/simulated_transactions_csv3/transactions.csv 

url ="/Users/arrao/Documents/Atchut/Book-PPML/Chapter7-FL/PS_20174392719_1491204439457_log.csv"
#url = "/Users/arrao/Documents/Atchut/Book-PPML/Chapter6/simulated_transactions_csv/2023-07-08.csv"
df_actual = pd.read_csv(url, sep=",")
df_actual

In [ ]:
df_transactions=df_actual.head(25000)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedShuffleSplit

print("No of Fraud Transactions:", df_transactions["isFraud"].value_counts()[0])
print("No of Non Fraud Transactions:", df_transactions["isFraud"].value_counts()[1])

print('No Frauds', round(df_transactions['isFraud'].value_counts()[0]/len(df_transactions) * 100,2), '% of the dataset')
print('Frauds', round(df_transactions['isFraud'].value_counts()[1]/len(df_transactions) * 100,2), '% of the dataset')


In [ ]:
df_transactions.dtypes

In [ ]:
from sklearn.preprocessing import LabelEncoder
encoder = {}
for i in df_transactions.select_dtypes('object').columns:
    encoder[i] = LabelEncoder()
    df_transactions[i] = encoder[i].fit_transform(df_transactions[i])

In [ ]:
X = df_transactions.drop('isFraud', axis=1)
y = df_transactions['isFraud']

In [ ]:
from imblearn.over_sampling import SMOTE

In [ ]:
over_sample = SMOTE(random_state=0)
X,y = over_sample.fit_resample(X,y)

In [ ]:
y.value_counts()

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X = scaler.fit_transform(X)

In [ ]:
X = df_transactions[['step', 'type', 'amount', 'nameOrig','oldbalanceOrg', 'newbalanceOrig','nameDest', 'oldbalanceDest', 'isFlaggedFraud']]
y= df_transactions['isFraud']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
# Turn the values into an array for feeding the classification algorithms.
X_train = X_train.values
X_test = X_test.values
y_train = y_train.values
y_test = y_test.values

In [ ]:
y_train

In [ ]:
y_test

In [ ]:
print(X_test)

In [ ]:
from typing import Tuple, Union, List
import numpy as np
from sklearn.linear_model import LogisticRegression
import openml

XY = Tuple[np.ndarray, np.ndarray]
Dataset = Tuple[XY, XY]
LogRegParams = Union[XY, Tuple[np.ndarray]]
XYList = List[XY]

def get_model_parameters(model: LogisticRegression) -> LogRegParams:
    """Returns the paramters of a sklearn LogisticRegression model."""
    if model.fit_intercept:
        params = [
            model.coef_,
            model.intercept_,
        ]
    else:
        params = [
            model.coef_,
        ]
    return params


def set_model_params(
    model: LogisticRegression, params: LogRegParams
) -> LogisticRegression:
    """Sets the parameters of a sklean LogisticRegression model."""
    model.coef_ = params[0]
    if model.fit_intercept:
        model.intercept_ = params[1]
    return model

def shuffle(X: np.ndarray, y: np.ndarray) -> XY:
    """Shuffle X and y."""
    rng = np.random.default_rng()
    idx = rng.permutation(len(X))
    return X[idx], y[idx]


def partition(X: np.ndarray, y: np.ndarray, num_partitions: int) -> XYList:
    """Split X and y into a number of partitions."""
    return list(
        zip(np.array_split(X, num_partitions), np.array_split(y, num_partitions))
    )

def set_initial_params(model: LogisticRegression):
    """Sets initial parameters as zeros Required since model params are
    uninitialized until model.fit is called.
    But server asks for initial parameters from clients at launch. Refer
    to sklearn.linear_model.LogisticRegression documentation for more
    information.
    """
    n_classes = 2  
    n_features = 9  # Number of features in dataset
    model.classes_ = np.array([i for i in range(n_classes)])

    model.coef_ = np.zeros((n_classes, n_features))
    if model.fit_intercept:
        model.intercept_ = np.zeros((n_classes,))


In [ ]:
import warnings
import flwr as fl
import numpy as np

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss

In [ ]:
partition_id = np.random.choice(10)
(X_train, y_train) = partition(X_train, y_train, 10)[partition_id]

In [ ]:
X_train

In [ ]:
partition_id

In [ ]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(
        penalty="l2",
        max_iter=1,  # local epoch
        warm_start=True,  # prevent refreshing weights when fitting
    )

model.fit(X_train, y_train)

In [ ]:
class AML_Detection_Client(fl.client.NumPyClient):
        def get_parameters(self, config):  # type: ignore
            return get_model_parameters(model)

        def fit(self, parameters, config):  # type: ignore
            set_model_params(model, parameters)
            # Ignore convergence failure due to low local epochs
            with warnings.catch_warnings():
                model.fit(X_train, y_train)
                warnings.simplefilter("ignore")
            print(f"Training finished for round {config['server_round']}")
            return get_model_parameters(model), len(X_train), {}

        def evaluate(self, parameters, config):  # type: ignore
            set_model_params(model, parameters)
            loss = log_loss(y_test, model.predict_proba(X_test))
            accuracy = model.score(X_test, y_test)
            print(loss,accuracy)
            return loss, len(X_test), {"accuracy": accuracy}

In [ ]:
fl.client.start_numpy_client(server_address="0.0.0.0:8080", client=AML_Detection_Client())